In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.0 MB/s eta 0:00:00


In [4]:
import pandas as pd
import tensorflow as tf
import keras
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM, BertForNextSentencePrediction, pipeline, BertForQuestionAnswering, BertForTokenClassification, BertForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [5]:
#email = pd.read_csv('../../../../../../data/r4.2/email.csv')
file = pd.read_csv('/content/drive/MyDrive/Praxis/data/file.csv')
#http = pd.read_csv('../../../../../../data/r4.2/http.csv')
#logon = pd.read_csv('../../../../../../data/r4.2/logon.csv')
#device = pd.read_csv('../../../../../../data/r4.2/device.csv')
#tweets = pd.read_csv('/content/drive/MyDrive/Praxis/code/tweets.csv')
#enron = pd.read_csv('../../../../../../data/praxis/ENRON/emails_enron.csv')


In [6]:
#files = [file,email,http,logon,device]

In [7]:
#for x in files:
#    print(x.shape)
file.shape

(445581, 6)

In [8]:
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
named_params = list(model.named_parameters())

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [11]:
tokenizer.encode_plus('Ken is the best!')

{'input_ids': [101, 6358, 2003, 1996, 2190, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [12]:
response = model(torch.tensor(tokenizer.encode('Ken is the best')).unsqueeze(0))

In [13]:
#response

In [14]:
response.pooler_output.shape

torch.Size([1, 768])

In [15]:
model.pooler

BertPooler(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (activation): Tanh()
)

In [16]:
cls_embedding = response.last_hidden_state[:, 0, :].unsqueeze(0)

In [17]:
cls_embedding.shape

torch.Size([1, 1, 768])

In [18]:
model.pooler(cls_embedding).shape

torch.Size([1, 768])

In [19]:
(model.pooler(cls_embedding) == response.pooler_output).all()

tensor(True)

In [20]:
'sinan' in tokenizer.vocab

False

In [21]:
i=0

In [22]:
encoded_file_content = tokenizer.encode_plus(file['content'][0])

In [23]:
encoded_file_content

{'input_ids': [101, 1040, 2692, 1011, 12935, 1011, 2340, 1011, 1041, 2692, 1011, 17350, 1011, 29491, 1011, 20720, 1011, 1041, 2487, 2076, 7669, 3452, 17138, 3904, 9048, 16173, 2102, 4711, 6048, 4105, 2011, 2738, 2104, 3356, 2520, 2019, 5955, 2261, 9576, 5590, 5002, 2153, 19817, 25438, 18083, 2071, 8003, 2205, 4415, 2309, 4672, 2185, 2219, 8489, 2068, 5941, 6470, 2114, 2035, 2646, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [24]:
#model.embeddings

In [25]:
example = "Ken is the best guy!"

In [26]:
tokenizer.encode(example, return_tensors='pt')

tensor([[ 101, 6358, 2003, 1996, 2190, 3124,  999,  102]])

In [27]:
model.embeddings.word_embeddings(tokenizer.encode(example, return_tensors='pt'))

tensor([[[ 0.0136, -0.0265, -0.0235,  ...,  0.0087,  0.0071,  0.0151],
         [-0.0098, -0.0581, -0.0190,  ..., -0.0698, -0.0208, -0.0665],
         [-0.0360, -0.0246, -0.0257,  ...,  0.0034, -0.0018,  0.0269],
         ...,
         [-0.0141, -0.1070,  0.0017,  ..., -0.0354, -0.0362, -0.0096],
         [ 0.0298, -0.0373, -0.0356,  ...,  0.0161,  0.0192,  0.0173],
         [-0.0145, -0.0100,  0.0060,  ..., -0.0250,  0.0046, -0.0015]]],
       grad_fn=<EmbeddingBackward0>)

In [28]:
model.embeddings.position_embeddings

Embedding(512, 768)

In [29]:
model.embeddings.position_embeddings(torch.LongTensor(range(9)))

tensor([[ 1.7505e-02, -2.5631e-02, -3.6642e-02,  ...,  3.3437e-05,
          6.8312e-04,  1.5441e-02],
        [ 7.7580e-03,  2.2613e-03, -1.9444e-02,  ...,  2.8910e-02,
          2.9753e-02, -5.3247e-03],
        [-1.1287e-02, -1.9644e-03, -1.1573e-02,  ...,  1.4908e-02,
          1.8741e-02, -7.3140e-03],
        ...,
        [ 6.4257e-03, -1.7664e-02, -2.2067e-02,  ...,  6.7531e-04,
          1.1108e-02,  3.7521e-03],
        [ 6.2613e-04, -1.6089e-02, -7.6365e-03,  ...,  5.3390e-03,
          1.5909e-02,  1.8119e-03],
        [ 2.0350e-03, -1.4569e-02, -1.4560e-02,  ...,  1.4465e-02,
          1.0401e-02,  3.7587e-03]], grad_fn=<EmbeddingBackward0>)

In [30]:
torch.LongTensor([0]*6)

tensor([0, 0, 0, 0, 0, 0])

In [31]:
model.embeddings.token_type_embeddings(torch.LongTensor([0]*6))

tensor([[ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
        [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
        [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
        [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
        [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
        [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086]],
       grad_fn=<EmbeddingBackward0>)

In [32]:
model.embeddings(tokenizer.encode(example, return_tensors='pt'))

tensor([[[ 0.1686, -0.2858, -0.3261,  ..., -0.0276,  0.0383,  0.1640],
         [ 0.3742, -0.3580, -0.1415,  ..., -0.3706,  0.5141, -0.9572],
         [-0.6270, -0.0633, -0.3143,  ...,  0.3427,  0.4636,  0.4594],
         ...,
         [-0.0221, -1.6964,  0.0810,  ..., -0.2988, -0.0200,  0.0699],
         [ 0.8539, -0.5522, -0.6580,  ...,  0.3174,  0.6846,  0.4836],
         [-0.2551, -0.2671,  0.0799,  ..., -0.4287,  0.3415, -0.1055]]],
       grad_fn=<NativeLayerNormBackward0>)

In [33]:
model.embeddings.LayerNorm(
    model.embeddings.word_embeddings(tokenizer.encode(example, return_tensors='pt')) +
    model.embeddings.position_embeddings(torch.LongTensor(range(8))) +
    model.embeddings.token_type_embeddings(torch.LongTensor([0]*8))
)

tensor([[[ 0.1686, -0.2858, -0.3261,  ..., -0.0276,  0.0383,  0.1640],
         [ 0.3742, -0.3580, -0.1415,  ..., -0.3706,  0.5141, -0.9572],
         [-0.6270, -0.0633, -0.3143,  ...,  0.3427,  0.4636,  0.4594],
         ...,
         [-0.0221, -1.6964,  0.0810,  ..., -0.2988, -0.0200,  0.0699],
         [ 0.8539, -0.5522, -0.6580,  ...,  0.3174,  0.6846,  0.4836],
         [-0.2551, -0.2671,  0.0799,  ..., -0.4287,  0.3415, -0.1055]]],
       grad_fn=<NativeLayerNormBackward0>)

In [34]:
model.embeddings(tokenizer.encode(example, return_tensors='pt')).size()

torch.Size([1, 8, 768])

In [35]:
bert_lm = BertForMaskedLM.from_pretrained('bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
#bert_lm

In [37]:
nlp = pipeline('fill-mask', model = 'bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [38]:
type(nlp.model)

transformers.models.bert.modeling_bert.BertForMaskedLM

In [39]:
nlp.tokenizer

BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [40]:
preds = nlp(f"If you dont {nlp.tokenizer.mask_token} at the sign, you will get a ticket.")

In [41]:
for p in preds:
  print(f"token: {p['token_str']}")

token: look
token: stop
token: glance
token: turn
token: stare


In [42]:
bert_nsp = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [43]:
#bert_nsp

In [44]:
text1 = "Ken is the man."
text2 = "He is the best as everything."

In [45]:
inputs = tokenizer(text1, text2, return_tensors='pt' )

In [46]:
inputs

{'input_ids': tensor([[ 101, 6358, 2003, 1996, 2158, 1012,  102, 2002, 2003, 1996, 2190, 2004,
         2673, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [47]:
outputs = bert_nsp(**inputs)

In [48]:
outputs

NextSentencePredictorOutput(loss=None, logits=tensor([[ 6.0411, -5.5872]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [49]:
outputs = bert_nsp(**inputs, labels=torch.LongTensor([0]))

In [50]:
outputs

NextSentencePredictorOutput(loss=tensor(8.9407e-06, grad_fn=<NllLossBackward0>), logits=tensor([[ 6.0411, -5.5872]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [51]:
outputs = bert_nsp(**inputs, labels=torch.LongTensor([1]))

In [52]:
outputs

NextSentencePredictorOutput(loss=tensor(11.6282, grad_fn=<NllLossBackward0>), logits=tensor([[ 6.0411, -5.5872]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [53]:
bert_sq = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [54]:
#bert_sq

In [55]:
bert_sq.classifier

Linear(in_features=768, out_features=3, bias=True)

In [56]:
finbert = pipeline(task="text-classification", model='ProsusAI/finbert', tokenizer='ProsusAI/finbert')

In [57]:
finbert("The year was 1929.")

[{'label': 'neutral', 'score': 0.8950798511505127}]

In [58]:
ner_example = pipeline(task='ner', model='bert-base-cased', tokenizer='bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [59]:
#ner_example("My name is Ken and I am from Miami")

In [60]:
bert_qa = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [61]:
#bert_qa

In [62]:
model_qa_squad2 = "deepset/roberta-base-squad2"

In [63]:
squad_qa = pipeline(task="question-answering", model=model_qa_squad2, tokenizer=model_qa_squad2)

In [64]:
sequence = "Where does Ken work?", "Ken is working for PowerSchool and John works for the FBI."

In [65]:
squad_qa(*sequence)

{'score': 0.8899106383323669, 'start': 19, 'end': 30, 'answer': 'PowerSchool'}

In [66]:
nlp = pipeline(task='fill-mask', model='distilbert-base-cased')

In [67]:
print(type(nlp.model))

<class 'transformers.models.distilbert.modeling_distilbert.DistilBertForMaskedLM'>


In [68]:
preds = nlp(f"if you don't {nlp.tokenizer.mask_token} at the sign, you will get a ticket")

In [69]:
for p in preds:
  print(f"token: {p['token_str']}, score: {100*p['score']:,.2f}%")

token: look, score: 58.75%
token: stop, score: 7.81%
token: glance, score: 4.35%
token: arrive, score: 2.23%
token: appear, score: 1.89%


We can average the hidden layer outputs to make our own vector instead of the actual pooler that comes with pytorch class.

In [70]:
from transformers import Trainer, TrainingArguments, DistilBertForSequenceClassification, DistilBertTokenizerFast, DataCollatorWithPadding 

In [71]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [72]:
from datasets import load_metric, Dataset

In [73]:
snips_file = pd.read_csv('/content/drive/MyDrive/Praxis/data/train.csv')

In [74]:
snips_file

,text,slot,intent
0,listen to westbam alumb allergic on google music,O O B-artist O B-album O B-service I-service,PlayMusic
1,add step to me to the 50 clásicos playlist,O B-entity_name I-entity_name I-entity_name O ...,AddToPlaylist
2,i give this current textbook a rating value of...,O O O B-object_select B-object_type O O O O B-...,RateBook
3,play the song little robin redbreast,O O B-music_item B-track I-track I-track,PlayMusic
4,please add iris dement to my playlist this is ...,O O B-artist I-artist O B-playlist_owner O B-p...,AddToPlaylist
...,...,...,...
13079,i want to eat choucroute at a brasserie for 8,O O O O B-served_dish O O B-restaurant_type O ...,BookRestaurant
13080,play funky heavy bluesy,O B-playlist I-playlist I-playlist,PlayMusic
13081,rate the current album 2 points out of 6,O O B-object_select B-object_type B-rating_val...,RateBook
13082,go to the photograph the inflated tear,O O O B-object_type B-object_name I-object_nam...,SearchCreativeWork


In [75]:
snips_file['intent'].size

13084

In [76]:
utterances = list(snips_file['text'])

In [77]:
tokenized_utterances = []
for j in utterances:
  k = j.split(" ")
  tokenized_utterances.append(k)

In [78]:
tokenized_utterances[0]

['listen', 'to', 'westbam', 'alumb', 'allergic', 'on', 'google', 'music']

In [79]:
labels_for_tokens = []
for j in list(snips_file['slot']) :
  k = j.split(" ")
  labels_for_tokens.append(k)

In [80]:
labels_for_tokens[3]

['O', 'O', 'B-music_item', 'B-track', 'I-track', 'I-track']

In [81]:
print(utterances[0])

listen to westbam alumb allergic on google music


In [82]:
sequence_labels = list(snips_file['intent'])

In [83]:
sequence_labels[0]

'PlayMusic'

In [84]:
len(sequence_labels), len(utterances), len(tokenized_utterances), len(labels_for_tokens)

(13084, 13084, 13084, 13084)

In [85]:
unique_sequence_labels = list(set(sequence_labels))

In [86]:
unique_sequence_labels

['AddToPlaylist',
 'SearchScreeningEvent',
 'PlayMusic',
 'GetWeather',
 'SearchCreativeWork',
 'RateBook',
 'BookRestaurant']

In [87]:
sequence_labels = [unique_sequence_labels.index(l) for l in sequence_labels]

In [88]:
from functools import reduce

In [89]:
unique_token_labels = list(set(reduce(lambda x, y: x + y, labels_for_tokens)))

In [90]:
labels_for_tokens = [[unique_token_labels.index(l) for l in j] for j in labels_for_tokens]

In [91]:
#unique_token_labels

In [92]:
sequence_labels[0]

2

In [93]:
unique_sequence_labels[sequence_labels[0]]

'PlayMusic'

In [94]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [95]:
from datasets import Dataset

In [96]:
snips_dataset = Dataset.from_dict(
    dict(
         utterance = utterances,
         lebel=sequence_labels,
         tokens = tokenized_utterances,
         token_labels = labels_for_tokens

    )
)
snips_dataset = snips_dataset.train_test_split(test_size=.2)

In [97]:
snips_dataset['train'][10]

{'utterance': 'add brian may to my reggae infusions list',
 'lebel': 0,
 'tokens': ['add', 'brian', 'may', 'to', 'my', 'reggae', 'infusions', 'list'],
 'token_labels': [3, 39, 13, 3, 56, 48, 23, 3]}

In [99]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [100]:
def preprocess_function(examples):
  return tokenizer(examples["utterance"], truncation=True)

In [101]:
seq_clf_tokenized_snips = snips_dataset.map(preprocess_function, batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [104]:
seq_clf_tokenized_snips['train'][0]

{'utterance': 'add this song to my playlist named wild country',
 'lebel': 0,
 'tokens': ['add',
  'this',
  'song',
  'to',
  'my',
  'playlist',
  'named',
  'wild',
  'country'],
 'token_labels': [3, 3, 60, 3, 56, 3, 3, 48, 23],
 'input_ids': [101,
  5587,
  2023,
  2299,
  2000,
  2026,
  2377,
  9863,
  2315,
  3748,
  2406,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [105]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [106]:
sequence_clf_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(unique_sequence_labels))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classi

In [107]:
sequence_clf_model.config.id2label = {i: l for i, l in enumerate(unique_sequence_labels)}

In [108]:
sequence_clf_model.config.id2label[0]

'AddToPlaylist'

In [109]:
metric = load_metric('accuracy')

<ipython-input-109-343a7ea47872>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')


In [110]:
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=labels)



In [111]:
epochs = 2

In [117]:
training_args = TrainingArguments(
    output_dir = "./snips_clf/results",
    num_train_epochs = epochs,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,

    warmup_steps = len(seq_clf_tokenized_snips['train']) // 5,
    weight_decay = .05,

    logging_steps=1,
    log_level='info',
    evaluation_strategy='epoch',
    save_strategy='epoch'

)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [118]:
trainer = Trainer(
    model = sequence_clf_model,
    args = training_args,
    train_dataset=seq_clf_tokenized_snips['train'],
    eval_dataset=seq_clf_tokenized_snips['test'],
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

In [119]:
trainer.evaluate()


***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, utterance, lebel, tokens. If token_labels, utterance, lebel, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_runtime': 86.0291,
 'eval_samples_per_second': 30.42,
 'eval_steps_per_second': 0.953}